# Calcul de conditionnement

In [137]:
def hilbertMatrice(n):
    return Matrix(RDF, [[1/(i+j-1) for i in range(1, n+1)] for j in range(1, n+1)])

def heatEqDiscretization1(n):
    M = 2*matrix.identity(RDF, n)
    for i in range(n-1):
        M[i, i+1] += -1
        M[i+1, i] += -1
    return M

def heatEqDiscretization2(n):
    M = 4*matrix.identity(RDF, n**2)
    for i in range(n**2-1):
        M[i, i+1] += -1
        M[i+1, i] += -1
    for i in range(n**2-n):
        M[i, i+n] += -1
        M[i+n, i] += -1

    return M

points1 = [(n, heatEqDiscretization1(n).condition(p=2)) for n in range(1,15)]
points2 = [(n, heatEqDiscretization2(n).condition(p=2)) for n in range(1,15)]
points3 = [(n, ln(hilbertMatrice(n).condition(p=2))) for n in range(1,15)]

points(points2, color='red') + points(points1, color='blue') + points(points3, color='green')

42.69110611493177

In [ ]:
def findBiggestPivot(A): 
    return max(enumerate(A.column(0)), key = lambda x: abs(x[1]))

def buildPermutationMatrix(size, r1, r2):
    M = Matrix.identity(QQ, size)
    M[r1, r1] = 0
    M[r2 ,r2] = 0
    M[r2, r1] = 1
    M[r1 , r2] = 1
    return M 

def eliminateBelow(A, pivotValue):
    L = [1] 
    for i in range(1, A.nrows()):
        coeff = A[i, 0]/pivotValue
        A[i] -= coeff*A[0]
        L.append(coeff)
    return L

def fillVectorWithZeros(vectorList, size): # Take a list of length decreasing vectors and fill them with zeros from the given side
    for i in range(len(vectorList)):
        zerosNeeded = (size - len(vectorList[i]))
        vectorList[i] = zerosNeeded * [0] + list(vectorList[i])
    return vectorList

def fillMatrices(matrixList, size): # Take a list of size decreasing matrices and fill them with identity blocs
    for i in range(len(matrixList)):
        blockSizeNeeded = (size - matrixList[i].nrows())
        matrixList[i] = block_diagonal_matrix(matrix.identity(blockSizeNeeded), matrixList[i])
    return matrixList 

def gaussianElim(A, P, L, U):
    if A.determinant() == 0: raise Exception('Non-invertible')
    #print(A, " before any swaps \n")

    (pivotIndex, pivotValue) = findBiggestPivot(A) # Find max of pivot values

    permutationMatrix = buildPermutationMatrix(A.nrows(), 0, pivotIndex)
    P.append(permutationMatrix)
    A = permutationMatrix*A

    #print(A, " after possible swaps \n")

    U.append(A[0])
    L.append(eliminateBelow(A, pivotValue))

    #print(A, " after elim L = ", L, "\n")
    if A.nrows() == 1: # Condition d'arrèt, on retourne la liste de lignes
        return (fillMatrices(P, len(U[0])), fillVectorWithZeros(L, len(U[0])), fillVectorWithZeros(U, len(U[0])))
    return gaussianElim(A.submatrix(1, 1), P, L, U)

M = Matrix(QQ, [[15, 7, 3, 4], [10, 2, 16, 2], [5, 8, 10, 7], [1, 2, 3, 5]])
(P, L, U) = gaussianElim(M, [], [], [])
prod(P.reverse()) * column_matrix(L) * matrix(U)


def methodTest():
    successes = [0, 0, 0]
    for i in range(1000):
        A = random_matrix(QQ, 3)
        while A.determinant() == 0:
            A = random_matrix(QQ, 3)

        P = prod(gaussianElim(A, [], [], [])[0])
        L = column_matrix(gaussianElim(A, [], [], [])[1])
        U = matrix(gaussianElim(A, [], [], [])[2])
        
        expected = A.LU("partial")
        if(P == expected[0]): successes[0] += 1
        if(L == expected[1]): successes[1] += 1
        if(U == expected[2]): successes[2] += 1

    print(successes)

# Méthodes directes

In [ ]:
def findBiggestPivot(A): 
    return max(enumerate(A.column(0)), key = lambda x: abs(x[1]))

def buildPermutationMatrix(size, r1, r2):
    M = Matrix.identity(QQ, size)
    M[r1, r1] = 0
    M[r2 ,r2] = 0
    M[r2, r1] = 1
    M[r1 , r2] = 1
    return M 

def eliminateBelow(A, pivotValue):
    L = [1] 
    for i in range(1, A.nrows()):
        coeff = A[i, 0]/pivotValue
        A[i] -= coeff*A[0]
        L.append(coeff)
    return L

def fillVectorWithZeros(vectorList, size): # Take a list of length decreasing vectors and fill them with zeros from the given side
    for i in range(len(vectorList)):
        zerosNeeded = (size - len(vectorList[i]))
        vectorList[i] = zerosNeeded * [0] + list(vectorList[i])
    return vectorList

def fillMatrices(matrixList, size): # Take a list of size decreasing matrices and fill them with identity blocs
    for i in range(len(matrixList)):
        blockSizeNeeded = (size - matrixList[i].nrows())
        matrixList[i] = block_diagonal_matrix(matrix.identity(blockSizeNeeded), matrixList[i])
    return matrixList 

def gaussianElim(A, P, L, U):
    if A.determinant() == 0: raise Exception('Non-invertible')
    #print(A, " before any swaps \n")

    (pivotIndex, pivotValue) = findBiggestPivot(A) # Find max of pivot values

    permutationMatrix = buildPermutationMatrix(A.nrows(), 0, pivotIndex)
    P.append(permutationMatrix)
    A = permutationMatrix*A

    #print(A, " after possible swaps \n")

    U.append(A[0])
    L.append(eliminateBelow(A, pivotValue))

    #print(A, " after elim L = ", L, "\n")
    if A.nrows() == 1: # Condition d'arrèt, on retourne la liste de lignes
        return (fillMatrices(P, len(U[0])), fillVectorWithZeros(L, len(U[0])), fillVectorWithZeros(U, len(U[0])))
    return gaussianElim(A.submatrix(1, 1), P, L, U)

M = Matrix(QQ, [[15, 7, 3, 4], [10, 2, 16, 2], [5, 8, 10, 7], [1, 2, 3, 5]])
(P, L, U) = gaussianElim(M, [], [], [])
prod(P.reverse()) * column_matrix(L) * matrix(U)





In [129]:
M = Matrix(QQ, [
    [1, 3, 1], 
    [5, 4, 1], 
    [1, 2, 1]
])
b = vector([1, 1, 0])
def findBiggestPivot(column , i): 
    L = []
    for j in range(i, len(column)):
        L.append((j, abs(column[j])))
    return max(L, key = lambda x: abs(x[1]))

def pivotGauss(A):
    size = A.nrows()

    for i in range(size): # Triangule la matrice
        bestPivotIndex = findBiggestPivot(A.column(i), i)[0]
        # Trouve le pivot le plus grand en valeur absolue de la colonne considérée
        
        if bestPivotIndex != i and i != size-1: 
            A.swap_rows(i, bestPivotIndex)
        pivot = A[i, i]

        # Change le pivot courant pour le plus grand en valeur absolue, sauf pour la dernière ligne

        for j in range(i + 1, size):
            coeff = -A[j, i]/pivot
            A[j] += coeff*A[i]
        # Elimine les valeurs en dessous du pivot

    for i in range(size):  # Remonte le système
        currentPivotIndex = size-i-1
        A[currentPivotIndex] = A[currentPivotIndex]/float(A[currentPivotIndex, currentPivotIndex]) 
        # Divise le coefficient diagonal pour avoir un 1 sur la diagonale.

        pivot = A[currentPivotIndex, currentPivotIndex]
        for j in range(currentPivotIndex):
            coeff = -A[j, currentPivotIndex]/pivot
            A[j] += coeff*A[currentPivotIndex]
        # Elimine les valeurs au dessus du pivot

    return A.column(size)

In [133]:
from sage.matrix.constructor import random_echelonizable_matrix
dimension = 10
matrix_space = sage.matrix.matrix_space.MatrixSpace(QQ, dimension, dimension)
# Espace des matrices aléatoires de test

def luMethodTest():
    successes = [0, 0, 0]
    for i in range(1000):
        A = random_matrix(QQ, 3)
        while A.determinant() == 0:
            A = random_matrix(QQ, 3)

        P = prod(gaussianElim(A, [], [], [])[0])
        L = column_matrix(gaussianElim(A, [], [], [])[1])
        U = matrix(gaussianElim(A, [], [], [])[2])
        
        expected = A.LU("partial")
        if(P == expected[0]): successes[0] += 1
        if(L == expected[1]): successes[1] += 1
        if(U == expected[2]): successes[2] += 1

def solveMethodTest():
    successes = 0
    for i in range(1000):
        A = random_echelonizable_matrix(matrix_space, rank=dimension, upper_bound=40)
        b = random_matrix(QQ, dimension, 1)

        found = pivotGauss(A.augment(b)) # Retourne le vecteur de notre méthode (ligne)
        foundCasted = Matrix([found[i] for i in range(len(found))]) # Retourne le vecteur de notre méthode (ligne)
        expected = A.solve_right(b) # Retourne le vecteur exact (colonne)

        error = float((foundCasted.transpose() - expected).norm())
        if(error < 0.1): successes += 1
    return successes
    
print(solveMethodTest())

1000
